<a href="https://colab.research.google.com/github/MateusAmorim/Python/blob/master/Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 50 kB/s 
     |████████████████████████████████| 199 kB 62.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=1fc6bcd4a56119c29542919489c3c60a1b7cffff94ba162ae945e7c9f969cdb7
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb  404  Not Found [IP: 91.189.91.39 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jdk-headless_8u312-b07-0ubuntu1~18.04_amd64.deb  404  Not Found [IP: 91.189.91.39 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?
^C


In [ ]:
# Configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# Torna o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [ ]:
# iniciar uma sessão local 
from pyspark.sql import SparkSession
#sc = SparkSession.builder.master('local[*]').getOrCreate()
spark = SparkSession.builder.appName("Introducao").getOrCreate()

In [ ]:
# Verify SparkContext
print(spark)

# Print Spark version
print(spark.version)

3.3.0


In [ ]:
# Print the tables in the catalog
print(spark.catalog.listTables())

[Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [ ]:
from google.colab import files
files.upload()

Saving USD_BRL_hist.csv to USD_BRL_hist (1).csv


{'USD_BRL_hist.csv': b'"Data","USD_BRL"\r\n"31.12.2019",4.0195\r\n"30.12.2019",4.0195\r\n"27.12.2019",4.046\r\n"26.12.2019",4.056\r\n"25.12.2019",4.0838\r\n"24.12.2019",4.0838\r\n"23.12.2019",4.0835\r\n"20.12.2019",4.1025\r\n"19.12.2019",4.0697\r\n"18.12.2019",4.0634\r\n"17.12.2019",4.0719\r\n"16.12.2019",4.0632\r\n"13.12.2019",4.1082\r\n"12.12.2019",4.0908\r\n"11.12.2019",4.1242\r\n"10.12.2019",4.147\r\n"09.12.2019",4.1425\r\n"06.12.2019",4.1414\r\n"05.12.2019",4.186\r\n"04.12.2019",4.208\r\n"03.12.2019",4.2066\r\n"02.12.2019",4.2234\r\n"29.11.2019",4.2367\r\n"28.11.2019",4.1902\r\n"27.11.2019",4.2638\r\n"26.11.2019",4.2332\r\n"25.11.2019",4.2268\r\n"22.11.2019",4.1966\r\n"21.11.2019",4.1932\r\n"20.11.2019",4.1965\r\n"19.11.2019",4.1945\r\n"18.11.2019",4.218\r\n"15.11.2019",4.1962\r\n"14.11.2019",4.1951\r\n"13.11.2019",4.1704\r\n"12.11.2019",4.1677\r\n"11.11.2019",4.1518\r\n"08.11.2019",4.1635\r\n"07.11.2019",4.1007\r\n"06.11.2019",4.0836\r\n"05.11.2019",3.9937\r\n"04.11.2019",4.0156\

In [ ]:
arquivo = "USD_BRL_hist.csv"
flights = spark\
        .read.format("csv")\
        .option("inferSchema", "True")\
        .option("header", "True")\
        .csv(arquivo)

In [ ]:
#Verificando o shape do pyspark dataframe
print((flights.count(), len(flights.columns)))

(2608, 2)


In [ ]:
flights.show(10)

+----------+-------+
|      Data|USD_BRL|
+----------+-------+
|31.12.2019| 4.0195|
|30.12.2019| 4.0195|
|27.12.2019|  4.046|
|26.12.2019|  4.056|
|25.12.2019| 4.0838|
|24.12.2019| 4.0838|
|23.12.2019| 4.0835|
|20.12.2019| 4.1025|
|19.12.2019| 4.0697|
|18.12.2019| 4.0634|
+----------+-------+
only showing top 10 rows



In [ ]:
flights.printSchema()

root
 |-- Data: string (nullable = true)
 |-- USD_BRL: double (nullable = true)



In [ ]:
from pyspark.sql.functions import col

In [ ]:
#Retirando as datas e passando colunas para tipos corretos.
flights = flights.\
        withColumn("new_Data", col("Data").cast("string")).drop("Data")

In [ ]:
#renomeando colunas
flights = flights.withColumnRenamed("new_Data","Data")

In [ ]:
#Registrando o dataframe em uma view temporária
flights.createOrReplaceTempView("flights")

query = "FROM flights SELECT * LIMIT 10"

# Selecionando as 10 primeiras linhas do dataset
flights10 = spark.sql(query)

# Print o resultado
flights10.show()

+-------+----------+
|USD_BRL|      Data|
+-------+----------+
| 4.0195|31.12.2019|
| 4.0195|30.12.2019|
|  4.046|27.12.2019|
|  4.056|26.12.2019|
| 4.0838|25.12.2019|
| 4.0838|24.12.2019|
| 4.0835|23.12.2019|
| 4.1025|20.12.2019|
| 4.0697|19.12.2019|
| 4.0634|18.12.2019|
+-------+----------+



In [ ]:
sqlDF = spark.sql("SELECT * FROM flights LIMIT 10")
sqlDF.show()

+-------+----------+
|USD_BRL|      Data|
+-------+----------+
| 4.0195|31.12.2019|
| 4.0195|30.12.2019|
|  4.046|27.12.2019|
|  4.056|26.12.2019|
| 4.0838|25.12.2019|
| 4.0838|24.12.2019|
| 4.0835|23.12.2019|
| 4.1025|20.12.2019|
| 4.0697|19.12.2019|
| 4.0634|18.12.2019|
+-------+----------+



In [ ]:
# Registtrando o dataframe como view global
flights.createGlobalTempView("flights1")

# A visão temporária global está vinculada a um banco de dados preservado pelo sistema `global_temp`
spark.sql("SELECT * FROM global_temp.flights LIMIT 10").show()

+-------+----------+
|USD_BRL|      Data|
+-------+----------+
| 4.0195|31.12.2019|
| 4.0195|30.12.2019|
|  4.046|27.12.2019|
|  4.056|26.12.2019|
| 4.0838|25.12.2019|
| 4.0838|24.12.2019|
| 4.0835|23.12.2019|
| 4.1025|20.12.2019|
| 4.0697|19.12.2019|
| 4.0634|18.12.2019|
+-------+----------+



In [ ]:
query = "SELECT Data, USD_BRL, COUNT(*) as N FROM flights GROUP BY Data, USD_BRL"

# Rodando a query
flight_counts = spark.sql(query)

In [ ]:
import pandas as pd

In [ ]:
# Convertendo o resultado para pandas
df = flights.toPandas()

In [ ]:
df.head()

,USD_BRL,Data
0,4.0195,31.12.2019
1,4.0195,30.12.2019
2,4.0460,27.12.2019
3,4.0560,26.12.2019
4,4.0838,25.12.2019


In [ ]:
# Criando pandas dataframe
arq = "USD_BRL_hist.csv"
pd_temp = pd.read_csv(arq)

In [ ]:
pd_temp.head()

,Data,USD_BRL
0,31.12.2019,4.0195
1,30.12.2019,4.0195
2,27.12.2019,4.0460
3,26.12.2019,4.0560
4,25.12.2019,4.0838


In [ ]:
# Cria spark_temp a partir de pd_temp
spark_temp = spark.createDataFrame(pd_temp)

In [ ]:
spark_temp.show(5)

+----------+-------+
|      Data|USD_BRL|
+----------+-------+
|31.12.2019| 4.0195|
|30.12.2019| 4.0195|
|27.12.2019|  4.046|
|26.12.2019|  4.056|
|25.12.2019| 4.0838|
+----------+-------+
only showing top 5 rows

